# Lab 3 - Parser Implementation

This lab starts by describing how to define custom headers in a P4 program. It then explains how to implement a simple parser that parses the defined headers. The lab further shows how to track the parsing states of a packet inside the software switch.

<img src="./labs_files/lab1/figs/fabric_topology.png" width="550px"><br>

# Background

## Program headers and definitions

For several decades, the networking industry operated in a bottom-up approach. At the bottom of the system are the fixed-function Application Specific Integrated Circuits (ASICs), which enforce protocols, features, and processes available in the switch. Programmers and operators are limited to these capabilities when building their systems. Consequently, systems have features defined by ASIC vendors that are rigid and may not fit the network operators’ needs. Programmable switches and P4 represent a disruption of the networking industry by enabling a top-down approach for the design of network applications. With this approach, the programmer or network operator can precisely describe features and how packets are processed in the ASIC, using a high-level language, P4.

With the Protocol Independent Switch Architecture (PISA), the programmer defines the headers and corresponding parser as well as actions executed in the match-action pipeline and the deparser. The programmer has the flexibility of defining custom headers (i.e., a header not standardized). Such capability is not available in non-programmable devices.


<img src="./labs_files/lab3/figs/ethernet.PNG" width="400px"><br>

<img src="./labs_files/lab3/figs/ipv4.PNG" width="600px"><br>

<img src="./labs_files/lab3/figs/ipv6.PNG" width="600px"><br>


Figure below shows an excerpt of a P4 program where the headers are defined. This is typically written at the top of the program before the parsing starts. We can see that the programmer defined a header corresponding to Ethernet (lines 11-15). The Ethernet header fields are shown in the first figure above.

<img src="./labs_files/lab3/figs/headers_defs.PNG" width="500px"><br>

The programmer also defined an IPv4 header (lines 26-40). The IPv4 header format is shown in the second figure above and the IPv6 header is shown in the third figure above.

The code starts by including the core.p4 file (line 1) which defines some common types and variables used in all P4 programs. For instance, the packet_in and packet_out extern types which represent incoming and outgoing packets, respectively, are declared in core.p4. Next, the v1model.p4 file is included (line 2) to define the V1Model architecture and all its externs used when writing P4 programs. Line 3 creates a 16-bit constant TYPE_IPV4 with the value 0x800. This means that TYPE_IPV4 can be used later in the P4 program to reference the value 0x800. The typedef declarations (lines 7 - 9) are used to assign alternative names to types. Subsequently, the headers and the metadata structs that will be used in the program are defined. These headers are customized depending on how the programmer wants the packets to be parsed. The program in Figure 4 defines the Ethernet header (lines 11-15) and the IPv4 header (lines 26-40). The declarations inside each header are usually written after referring to the standard specifications of the protocol. Note in the ethernet_t header the macAddr_t is used rather than using a 48-bit field. Lines 17 - 19 show how to declare user-defined metadata, which are passed from one block to another as the packet propagates through the architecture. For simplicity, this program does not require any user metadata. 


## Programmable parser

The programmable parser permits the programmer to describe how the switch will process the packet. The parser de-encapsulates the headers, converting the original packet into a parsed representation of the packet. The parser can be represented as a state machine without cycles (direct acyclic graph), with one initial state (start) and two final states (accept or reject).

<img src="./labs_files/lab3/figs/parser1.PNG" width="450px"><br>

<img src="./labs_files/lab3/figs/parser2.PNG" width="500px"><br>

The first figure above shows the graphical representation of the parser and the second figure its corresponding P4 code. Note that packet is an instance of the packet_in extern (specific to V1Model) and is passed as a parameter to the parser. The extract method associated with the packet extracts N bits, where N is the total number of bits defined in the corresponding header (for example, 112 bits for Ethernet). Afterwards, the etherType field of the Ethernet header is examined using the select statement, and the program branches to the parse_ipv4 state if the etherType field corresponds to IPv4. The state transitions to the reject if it is not an IPv4 header, as shown in the figure above (Line 12). In the parse_ipv4 state, the IPv4 header is extracted, and the program unconditionally transitions to the accept state.


# Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the Experiment Slice

The following creates three node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "lab3"

In [ ]:
slice = fablib.new_slice(name="lab3")

### Step 3.2: Define the sites
The code below requests three sites from FABRIC: GATECH, STAR, and NCSA

<img src="./labs_files/lab1/figs/fabric_sites.png" width="40%"><br>

In [ ]:
site1='MICH'
site2='STAR'
site3='NCSA'

print (f'The selected sites are {site1}, {site2}, {site3}') 

### Step 3.3: Creating the nodes
The code below creates three nodes: server1, switch, and server2. The servers (server1 and server2) use the following
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1 and server3 will be created in site3

<img src="./labs_files/lab1/figs/creating_nodes.PNG" width="550px"><br>

In [ ]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

<img src="./labs_files/lab1/figs/adding_nics.PNG" width="550px"><br>

In [ ]:
server1_iface = server1.add_component(model='NIC_Basic').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_Basic').get_interfaces()[0]

### Step 3.5: Creating a node for the P4 switch
The code below creates a node that will run the P4 switch. The node use the following
<ul>
    <li> 16 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 40GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

The node will be created in site2

<img src="./labs_files/lab1/figs/adding_switch.PNG" width="550px"><br>

In [ ]:
# Add a node
switch = slice.add_node(name="switch", 
                      site=site2, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

### Step 3.6: Adding two interfaces to the switch
The code below adds two Network Interface Cards (NICs) to the switch.

<img src="./labs_files/lab1/figs/adding_switch_ports.PNG" width="550px"><br>

In [ ]:
switch_iface1 = switch.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch_iface2 = switch.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]

### Step 3.7: Connecting site1 and site2
Create a site-to-site network between site1 and site2 connecting server1 and the P4 switch

<img src="./labs_files/lab1/figs/connecting_nodes_server1_switch.PNG" width="550px"><br>

In [ ]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, switch_iface1])

### Step 3.8: Connecting site2 and site3
Create a site-to-site network between site2 and site3 connecting the P4 switch and server2

<img src="./labs_files/lab1/figs/connecting_nodes_server2_switch.PNG" width="550px"><br>

In [ ]:
net2 = slice.add_l2network(name='net2', interfaces=[switch_iface2, server2_iface])

### Step 3.9: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [ ]:
#Submit Slice Request
slice.submit();

# Step 4: Installing the required packages
In this step, we will install the required packages to run the labs. Specifically, we will install the BMv2 software switch and its control plane, the P4 compiler (p4c), and net-tools. 

## Step 4.1 Installing BMv2
The BMv2 software switch will be installed on the switch node. We will upload the script [scripts/install_bmv2.sh](./scripts/install_bmv2.sh) to the switch and execute it

In [ ]:
switch = slice.get_node(name="switch")     
switch.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

## Step 4.2 Installing net-tools
The net-tools package will be installed on the switch, server1 and server2 nodes. This package will allow us to use the ifconfig and the arp commands 

In [ ]:
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch.execute(f'sudo apt-get install -y net-tools', quiet=True)

## Step 4.3 Installing scapy
Installing scapy to be able to craft and send raw packets on the servers


In [ ]:
stdout, stderr = server1.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)

# Step 5: Assigning IP and MAC addresses
In this step, we will assign IPv4 addresses to the interfaces of the servers and the switch. We will also hardcode the MAC addresses. 

## Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./labs_files/lab1/figs/interfaces.PNG" width="550px"><br>

In [ ]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()
print(f'server1_iface: {server1_iface_name}')

node2_iface = server2.get_interface(network_name='net2') 
server2_iface_name = node2_iface.get_device_name()
print(f'server2_iface: {server2_iface_name}')

switch_iface1 = switch.get_interface(network_name='net1') 
switch_iface1_name = switch_iface1.get_device_name()
print(f'switch_iface1: {switch_iface1_name}')

switch_iface2 = switch.get_interface(network_name='net2') 
switch_iface2_name = switch_iface2.get_device_name()
print(f'switch_iface2: {switch_iface2_name}')

## Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

<img src="./labs_files/lab1/figs/interfaces_up.PNG" width="550px"><br>

In [ ]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface1_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface2_name} up', quiet=True)

## Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> server1_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>switch_iface1_MAC = '00:00:00:00:00:02' (shown as 00:02 in the figure below)</li>
    <li>switch_iface2_MAC = '00:00:00:00:00:03' (shown as 00:03 in the figure below)</li>
    <li>server2_iface_MAC = '00:00:00:00:00:04' (shown as 00:04 in the figure below)</li>
</ul>

<img src="./labs_files/lab1/figs/mac_addresses.PNG" width="550px"><br>

In [ ]:
server1_iface_MAC = '00:00:00:00:00:01'
switch_iface1_MAC = '00:00:00:00:00:02'
switch_iface2_MAC = '00:00:00:00:00:03'
server2_iface_MAC = '00:00:00:00:00:04'

## Step 5.4 Configuring the IP and MAC addresses on server1_iface and switch_iface1

We will use the network 192.168.1.0/24 between Site1 and Site2. We will assign the IP address 192.168.1.10 to server1's interface and 192.168.1.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_1.PNG" width="550px"><br>

In [ ]:
server1 = slice.get_node(name="server1")     

server1_switch_subnet = "192.168.1.0/24"
server1_ip = '192.168.1.10/24'
switch_ip1 = '192.168.1.1/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} {switch_ip1}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} hw ether {switch_iface1_MAC}')

## Step 5.5: Configuring the IP and MAC addresses on switch_iface2 and server2_iface

We will use the network 192.168.2.0/24 between Site2 and Site3. We will assign the IP address 192.168.2.10 to server2's interface and 192.168.2.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_2.PNG" width="550px"><br>

In [ ]:
server2 = slice.get_node(name="server2")     

server2_switch_subnet = "192.168.2.0/24"
server2_ip = '192.168.2.10/24'
switch_ip2 = '192.168.2.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} {switch_ip2}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} hw ether {switch_iface2_MAC}')

# Step 6: Configure forwarding and routing

## Step 6.1: Enable forwarding on the switch

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

The command will be executed on the switch device.

In [ ]:
command = 'sudo sysctl -w net.ipv4.ip_forward=1' 
stdout, stderr = switch.execute(command, quiet=True)

## Step 6.2: Delete routing entries for the routes to force traffic to go through the BMv2 switch

In this step, we are deleting the routes on the switch's routing table in Linux. By deleting the routes, the packets will go through the BMv2 switch instead of being forwarded by the kernel

In [ ]:
stdout, stderr = switch.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)

## Step 6.3: Configure routing

In this step, we will configure static routes on server1 and server2. 
<ul>
    <li> For server1, we will add a route to reach the network 192.168.2.0/24 via 192.168.1.1 </li>
    <li> For server2, we will add a route to reach the network 192.168.1.0/24 via 192.168.2.1 </li>
</ul>

<img src="./labs_files/lab1/figs/routing.PNG" width="550px"><br>

In [ ]:
gw1 = switch_ip1.split('/')[0]
gw2 = switch_ip2.split('/')[0]
stdout, stderr = server1.execute(f'sudo ip route add {server2_switch_subnet} via {gw1}')
stdout, stderr = server2.execute(f'sudo ip route add {server1_switch_subnet} via {gw2}')

## Step 6.4: Configure ARP

In this step, we will configure static ARP entries on server1 and server2. The reason we are doing this is because the switch does not process ARP packets unless programmed to. To make sure that ARP packets are not sent towards the switch, we will hardcode the MACs on the servers.

For each server, we will add an ARP entry to its switch's neighboring interface.

In [ ]:
stdout, stderr = server1.execute(f'sudo arp -s {gw1} {switch_iface1_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {gw2} {switch_iface2_MAC}')

# Step 7: Describing the headers
   
This section demonstrates how to define custom headers in a P4 program. It also shows how to use constants and typedefs to make the program more readable.


# Step 7.1: Listing header’s definitions into the headers.p4 file

Click on [headers.p4](./labs_files/lab3/src/headers.p4) to open the file in the editor.

<img src="./labs_files/lab3/figs/headers_prog.PNG" width="550px"><br>

We can see that the headers.p4 is empty and we have to fill it.

<hr>

We will start by defining some typedefs and constants. Write the following in the headers.p4 file

    typedef bit<48> macAddr_t;
    typedef bit<32> ip4Addr_t;
    const bit<16> TYPE_IPV4 = 0x800;

<img src="./labs_files/lab3/figs/headersp4.png" width="550px"><br>

In the figure above the typedef declarations used (lines 2 - 3) are used to assign alternative names to types. Here we are saying that macAddr_t can be used instead of bit<48>, and ip4Addr_t instead of bit<32>. We will use those typedefs when defining the headers. Line 4 shows how to define a constant with the name TYPE_IPV4 and a value of 0x800. We will use this value in the parser implementation. 

<hr>

Now we will define the Ethernet header. Add the following code to the headers.p4 file

    header ethernet_t {
        macAddr_t dstAddr;
        macAddr_t srcAddr;
        bit<16> etherType;
    }

<img src="./labs_files/lab3/figs/headersethernet.png" width="550px"><br>

Note how we used the typedef macAddr_t which corresponds to bit<48> when defining the destination MAC address field (dstAddr) and the source MAC address field (srcAddr). 

<hr>

Now we will define the IPv4 header. Add the following to the headers.p4 file

<img src="./labs_files/lab3/figs/headersipv4.png" width="550px"><br>

Consider the figure above. Note how we used the typedef ip4Addr_t which corresponds to bit<32> when defining the source IP address field (srcAddr) and the destination IP address field (dstAddr). Also, note how we are mapping the fields to those defined in the standard IPv4 header.

<hr>

Now we will create a struct to represent our metadata. Metadata are passed from one block to another as the packet propagates through the architecture. For simplicity, this program does not require any user metadata, and hence we will define it as empty with no fields. Add the following to the headers.p4 file:

    struct metadata {
    /* empty */
    }

<img src="./labs_files/lab3/figs/headersmetadata.png" width="550px"><br>

<hr>

Now we will create a struct to contain our headers (Ethernet and IPv4). Append the following code to the headers.p4 file

    struct headers {
        ethernet_t   ethernet;
        ipv4_t       ipv4;
    }

<img src="./labs_files/lab3/figs/headerscombined.png" width="550px"><br>

<hr>

**Save the changes by pressing Ctrl+s**.

# Step 8: Parser Implementation

Now it is time to define how the parser works. 

Click on [parser.p4](./labs_files/lab3/src/parser.p4) to open the file in the editor.
  
<img src="./labs_files/lab3/figs/parserempty.png" width="550px"><br>

We can see that the headers.p4 file that we just filled is included here in the parser. The file also includes a starter code which declares a parser named MyParser. Note how the headers and the metadata structs that we defined previously are passed as parameters to the parser. 

<hr>

Add the start state inside the parser by inserting the following code:

    state start {
         transition parse_ethernet;
    }

<img src="./labs_files/lab3/figs/parserstart.png" width="550px"><br>

The start state is the state where the parser begins parsing the packet. Here we are transitioning unconditionally to the parse_ethernet state.

<hr>

Add the parse_ethernet state inside the parser by inserting the following code:

  state parse_ethernet {
       packet.extract(hdr.ethernet);
       transition select(hdr.ethernet.etherType) {
          TYPE_IPV4: parse_ipv4;
          default: accept;
       }
  }
  
<img src="./labs_files/lab3/figs/parserethernet.png" width="550px"><br>

The parse_ethernet state extracts the Ethernet header and checks for the value of the header field etherType. Note how we reference a header field by specifying the header to which that field belongs (i.e., hdr.ethernet.etherType). If the value of etherType is TYPE_IPV4 (which corresponds to 0x800 as defined previously), the parser transitions to the parse_ipv4 state. Otherwise, the execution of the parser terminates.

<hr>

Add the parse_ipv4 state inside the parser by inserting the following code.

    state parse_ipv4 {
        packet.extract(hdr.ipv4);
        transition accept;
    }
<img src="./labs_files/lab3/figs/parseripv4.png" width="550px"><br>

The parse_ipv4 state extracts the IPv4 header and terminates the execution of the parser.

**Save the changes by pressing Ctrl+s**.

# Step 9: Uploading and running the P4 program on the switch

In this step, we upload the P4 program to the switch, compile it, and start the switch daemon. 

## Step 9.1: Uploading the P4 program

The P4 program [basic.p4](labs_files/lab3/src/basic.p4) is located under lab_files/lab3/src.

We will be uploading the whole directory since it includes other P4 files. 

In [ ]:
switch = slice.get_node(name='switch')        
switch.upload_directory('labs_files/lab3/src', '/home/ubuntu/lab3')

## Step 9.2: Compiling the P4 program

In this step, we will use the p4c compiler to compile the program.

Launch a new terminal by clicking on "File" -> "New" -> "Terminal".

<img src="./labs_files/lab3/figs/terminal.gif" width="600px"><br>

Copy the output of the command below and paste into the terminal to enter to the switch.

In [ ]:
switch.get_ssh_command()

Run the following commands in the terminal:
    
    sudo su
    p4c lab3/src/basic.p4
    simple_switch -i 0@ens8 -i 1@ens7 basic.json --log-console
    
<img src="./labs_files/lab3/figs/daemon.png" width="70%"><br>

## Step 9.3: Populating table from the control plane

In this step we will populate the forwarding table by executing a script. We will learn how to populate the tables manually in another lab.

In [ ]:
switch.upload_file('labs_files/lab3/rules.sh', 'rules.sh')
stdout, stderr = switch.execute('chmod +x rules.sh && ./rules.sh')

## Step 9.4: Sending a packet from server1 to the switch

In this step, we will send a packet from server1 to server2. The packet will be processed in the switch. 

In [ ]:
server1.upload_file('labs_files/lab3/src/send.py', 'send.py')
#server1.execute(f'sudo python3 send.py {server1_iface} 192.168.1.10 192.168.2.10 HelloWorld')
stdout, stderr = server1.execute(f'sudo python3 send.py ens7 192.168.1.10 192.168.2.10 HelloWorld')

## Step 9.5: Inspect the logs on the switch

Go back to the switch terminal and inspect the logs.

<img src="./labs_files/lab3/figs/parserlogipv4.png" width="750px"><br>

The figure above shows that the Ethernet and IPv4 header are extracted.

# Step 10: Augmenting the P4 program to parse IPv6

Now we will augment the program to parse IPv6 packets. 

Go back to the [headers.p4](./labs_files/lab3/src/headers.p4) file and add the following constant definition:
    
    const bit<16> TYPE_IPV6 = 0x86dd;

<img src="./labs_files/lab3/figs/constipv6.png" width="550px"><br>

<hr>

Add the IPv6 header definition as shown below:
    
    header ipv6_t{
        bit<4> version;
        bit<8> trafficClass;
        bit<20> flowLabel;
        bit<16> payloadLen;
        bit<8> nextHdr;
        bit<8> hopLimit;
        bit<128> srcAddr;
        bit<128> dstAddr;
    }
    
<img src="./labs_files/lab3/figs/headeripv6.png" width="550px"><br>

<hr>

Append the IPv6 header to the header’s data structure:
    
    ipv6_t ipv6;

<img src="./labs_files/lab3/figs/headeripv6struct.png" width="550px"><br>

<hr>

Go to the [parser.p4](./labs_files/lab3/src/parser.p4) file and add the following line to the parse_ethernet state.

<img src="./labs_files/lab3/figs/typeipv6.png" width="550px"><br>

<hr>

Add the parse_ipv6 state inside the parser by inserting the following code:
    
    state parse_ipv6 {
        packet.extract(hdr.ipv6);
        transition accept;
    }

<img src="./labs_files/lab3/figs/parseripv6.png" width="550px"><br>

**Save the changes by pressing Ctrl+s.**


## Step 10.1: Uploading the P4 program

The P4 program [basic.p4](labs_files/lab3/src/basic.p4) is located under lab_files/lab3/src.

We will be uploading the whole directory since it includes other P4 files. 

In [ ]:
switch = slice.get_node(name='switch')        
switch.upload_directory('labs_files/lab3/src', '/home/ubuntu/lab3')

In the switch's terminal, press Ctrl+C to terminate the execution of the previous instance of the switch, then run the following commands:
    
    p4c lab3/src/basic.p4
    simple_switch -i 0@ens8 -i 1@ens7 basic.json --log-console
    
<img src="./labs_files/lab3/figs/daemon2.png" width="750px"><br>

## Step 10.2: Populating table from the control plane

In this step we will populate the forwarding table by executing a script. We will learn how to populate the tables manually in another lab.

In [ ]:
stdout, stderr = switch.execute('chmod +x rules.sh && ./rules.sh')

## Step 10.3: Sending an IPv6 packet from the host

In this step, we will send an IPv6 packet from server1 to server2. The packet will be processed in the switch. 

In [ ]:
server1.upload_file('labs_files/lab3/src/send_ipv6.py', 'send.py')
#server1.execute('sudo python3 send.py {server1_iface} 192.168.1.10 192.168.2.10 HelloWorld')
stdout, stderr = server1.execute(f'sudo python3 send.py ens7')

## Step 10.4: Inspect the logs on the switch

Go back to the switch terminal and inspect the logs.

<img src="./labs_files/lab3/figs/parserlogipv6.png" width="750px"><br>

The figure above shows that the Ethernet and IPv6 headers are extracted.

## Step 11: Delete the Slice

This concludes Lab 3. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="lab3")
slice.delete()